---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [199]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [230]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [265]:
def StateCode(StateName):
    return states[StateName]

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"] )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    RegionList = list()
    file = open('university_towns.txt')

    for line in file: 
        #print(line)
        if '[edit]' in line:
            State = re.findall('(.*)\[',line)[0]       
        else:
            #RegionName = re.search('(.*?)(( \()|(\, )|:)', line) #.group(1)
            RegionName = re.search('(.*?)(( \())', line) #.group(1)
            if RegionName != None:
                RegionList.append({'State': State,'RegionName': RegionName.group(1)})
            else:
                RegionList.append({'State': State,'RegionName': line.replace('\n','')})
                
    df = pd.DataFrame(RegionList)[['State','RegionName']]            
    
    return df

get_list_of_university_towns()

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(get_list_of_university_towns())


State                    RegionName
0          Alabama                        Auburn
1          Alabama                      Florence
2          Alabama                  Jacksonville
3          Alabama                    Livingston
4          Alabama                    Montevallo
5          Alabama                          Troy
6          Alabama                    Tuscaloosa
7          Alabama                      Tuskegee
8           Alaska                     Fairbanks
9          Arizona                     Flagstaff
10         Arizona                         Tempe
11         Arizona                        Tucson
12        Arkansas                   Arkadelphia
13        Arkansas                        Conway
14        Arkansas                  Fayetteville
15        Arkansas                     Jonesboro
16        Arkansas                      Magnolia
17        Arkansas                    Monticello
18        Arkansas                  Russellville
19        Arkansas                        Searcy
20      California                        Angwin
21      California                        Arcata
22      California                      Berkeley
23      California                         Chico
24      California                     Claremont
25      California                        Cotati
26      California                         Davis
27      California                        Irvine
28      California                    Isla Vista
29      California  University Park, Los Angeles
..             ...                           ...
487       Virginia                          Wise
488       Virginia                    Chesapeake
489     Washington                    Bellingham
490     Washington                        Cheney
491     Washington                    Ellensburg
492     Washington                       Pullman
493     Washington  University District, Seattle
494  West Virginia                        Athens
495  West Virginia                    Buckhannon
496  West Virginia                      Fairmont
497  West Virginia                     Glenville
498  West Virginia                    Huntington
499  West Virginia                    Montgomery
500  West Virginia                    Morgantown
501  West Virginia                 Shepherdstown
502  West Virginia                  West Liberty
503      Wisconsin                      Appleton
504      Wisconsin                    Eau Claire
505      Wisconsin                     Green Bay
506      Wisconsin                     La Crosse
507      Wisconsin                       Madison
508      Wisconsin                     Menomonie
509      Wisconsin                     Milwaukee
510      Wisconsin                       Oshkosh
511      Wisconsin                   Platteville
512      Wisconsin                   River Falls
513      Wisconsin                 Stevens Point
514      Wisconsin                      Waukesha
515      Wisconsin                    Whitewater
516        Wyoming                       Laramie

[517 rows x 2 columns]

In [232]:
GDPdf = pd.read_excel('gdplev.xls',skiprows=2, header=3)[2:]
GDPdf = GDPdf.iloc[212:][['Unnamed: 4','GDP in billions of chained 2009 dollars.1']].reset_index(drop=True)
GDPdf.columns = ['Quarter','GDP']

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP1 = 0
    GDP2 = 0
    StartQuarter = ''
    for i in range(len(GDPdf)):
        if GDP1 == 0:
            GDP1 = GDPdf.iloc[i].GDP
        elif GDP2 == 0:
            GDP2 = GDPdf.iloc[i].GDP
        else:
            if int(GDP2)<int(GDP1) and GDPdf.iloc[i].GDP<(GDP2):
                return StartQuarter
            GDP1=GDP2
            GDP2=GDPdf.iloc[i].GDP
            StartQuarter = GDPdf.iloc[i].Quarter 
    return "" #2008q3

get_recession_start()

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(GDPdf)

'2008q3'

In [233]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP1 = 0
    GDP2 = 0
    StartQuarter = get_recession_start()
    for i in range(len(GDPdf)):
        if GDP1 == 0:
            GDP1 = GDPdf.iloc[i].GDP
        elif GDP2 == 0:
            GDP2 = GDPdf.iloc[i].GDP
        else:
            if int(GDP2) > int(GDP1) and GDPdf.iloc[i].GDP>(GDP2) and GDPdf.iloc[i].Quarter > StartQuarter:
                return GDPdf.iloc[i].Quarter
            GDP1=GDP2
            GDP2=GDPdf.iloc[i].GDP       
    
    return "" #2009q4
get_recession_end()

'2009q4'

In [234]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    Bottom_GDP = 0
    BottomQuarter = ''
    StartQuarter = get_recession_start()
    EndQuarter = get_recession_end()
    for i in range(len(GDPdf)):
        if GDPdf.iloc[i].Quarter >= StartQuarter and GDPdf.iloc[i].Quarter <= EndQuarter:
            if Bottom_GDP == 0:
                Bottom_GDP = GDPdf.iloc[i].GDP
                BottomQuarter = GDPdf.iloc[i].Quarter 
            elif GDPdf.iloc[i].GDP < Bottom_GDP:
                Bottom_GDP = GDPdf.iloc[i].GDP
                BottomQuarter = GDPdf.iloc[i].Quarter            
    return BottomQuarter

get_recession_bottom()

'2009q2'

In [235]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv',)
    df2 = pd.DataFrame()
    
    ColList=['State','RegionName']
    df2[ColList] = df[ColList]
    
    for colname in df.columns[6:] :
        year = int(colname[:4])
        month = int(colname[5:7])
        if month>=1 and month<4:
            quarter = 1
        elif month >=4 and month<7:
            quarter = 2
        elif month >=7 and month<10:
            quarter = 3
        else:
            quarter = 4
        NewColName = str(year)+'q'+str(quarter)
        df.rename(columns={colname:NewColName}, inplace=True)
        if year>=2000 and NewColName not in ColList:
            ColList+=[NewColName]
        
    df = df[ColList]

    for colname in df.columns.unique()[2:]:
        df2[colname] = df[colname].mean(axis=1)
        
    df2['State'] = df2['State'].apply(StateCode)   
    
    df2 = df2.set_index(['State','RegionName'])
    return df2

#convert_housing_data_to_quarters()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
# print()

#convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

In [236]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recessionStart = get_recession_start()
    recessionEnd = get_recession_end()
    recessionBottom = get_recession_bottom()
    
    if recessionStart[5]==1:
        beforeRecession = str(int(recessionStart[:4])-1)+'q4'
    else:
        beforeRecession = recessionStart[:5]+str(int(recessionStart[5])-1)
    
    UniTowns = get_list_of_university_towns()
    UniTowns['ItsUni'] = True
    
    HouseData = convert_housing_data_to_quarters()
    HouseData['PriceRateChange'] = HouseData[beforeRecession]/HouseData[recessionBottom]

    
    df = pd.merge(HouseData, UniTowns, how = 'left', left_index=True, right_on = ['State','RegionName'])
    df['ItsUni'] = df['ItsUni'].fillna(False)
    
    df = df[['ItsUni','2008q3','2009q2','PriceRateChange']].dropna() 
    
    NonUni = df[df['ItsUni']==False]
    Uni = df[df['ItsUni']==True]
    
    res = ttest_ind(NonUni['PriceRateChange'],Uni['PriceRateChange'])
    
    if NonUni['PriceRateChange'].mean(axis=0) < Uni['PriceRateChange'].mean(axis=0):
        better="non-university town"
    else:
        better="university town"
    
    return (res.pvalue<0.01, res.pvalue, better) 

run_ttest()
#(True, 0.0027678022717951317, 'university town')

(True, 0.0027678022717951317, 'university town')